In [1]:
# train a Deep Learning model on MNIST Data to recognize digits in images using pytorch
# using the context manager

In [2]:
# import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from codecarbon import EmissionsTracker

In [3]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
# set the batch size
batch_size = 64

In [6]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# creating model
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [8]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
# import the tracker using Context Manager
with EmissionsTracker() as tracker:
    epochs = 10
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

[codecarbon INFO @ 19:19:01] [setup] RAM Tracking...
[codecarbon INFO @ 19:19:01] [setup] GPU Tracking...
[codecarbon INFO @ 19:19:01] No GPU found.
[codecarbon INFO @ 19:19:01] [setup] CPU Tracking...
[codecarbon WARNING @ 19:19:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:19:01] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 19:19:01] >>> Tracker's metadata:
[codecarbon INFO @ 19:19:01]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 19:19:01]   Python version: 3.8.18
[codecarbon INFO @ 19:19:01]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 19:19:01]   Available RAM : 16.000 GB
[codecarbon INFO @ 19:19:01]   CPU count: 8
[codecarbon INFO @ 19:19:01]   CPU model: Apple M1
[codecarbon INFO @ 19:19:01]   GPU count: None
[codecarbon INFO @ 19:19:01]   GPU model: None


Epoch 1
-------------------------------
loss: 2.299253  [   64/60000]
loss: 2.298460  [ 6464/60000]
loss: 2.301398  [12864/60000]
loss: 2.275913  [19264/60000]
loss: 2.285846  [25664/60000]
loss: 2.283754  [32064/60000]
loss: 2.275487  [38464/60000]
loss: 2.282565  [44864/60000]
loss: 2.265837  [51264/60000]
loss: 2.257737  [57664/60000]
Test Error: 
 Accuracy: 32.6%, Avg loss: 2.258804 

Epoch 2
-------------------------------
loss: 2.253422  [   64/60000]
loss: 2.250460  [ 6464/60000]
loss: 2.263000  [12864/60000]
loss: 2.214252  [19264/60000]
loss: 2.237579  [25664/60000]
loss: 2.235057  [32064/60000]
loss: 2.214083  [38464/60000]
loss: 2.236570  [44864/60000]
loss: 2.203008  [51264/60000]
loss: 2.190180  [57664/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 2.193906 

Epoch 3
-------------------------------
loss: 2.187273  [   64/60000]
loss: 2.179679  [ 6464/60000]
loss: 2.205188  [12864/60000]
loss: 2.121718  [19264/60000]
loss: 2.162431  [25664/60000]
loss: 2.157169  [32064/600

[codecarbon INFO @ 19:19:19] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:19:19] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:19:19] 0.000046 kWh of electricity used since the beginning.


loss: 2.100945  [51264/60000]
loss: 2.080391  [57664/60000]
Test Error: 
 Accuracy: 61.9%, Avg loss: 2.085522 

Epoch 4
-------------------------------
loss: 2.077645  [   64/60000]
loss: 2.060684  [ 6464/60000]
loss: 2.104486  [12864/60000]
loss: 1.971674  [19264/60000]
loss: 2.031790  [25664/60000]
loss: 2.019021  [32064/60000]
loss: 1.951364  [38464/60000]
loss: 2.021792  [44864/60000]
loss: 1.928763  [51264/60000]
loss: 1.899580  [57664/60000]
Test Error: 
 Accuracy: 68.3%, Avg loss: 1.900540 

Epoch 5
-------------------------------
loss: 1.896494  [   64/60000]
loss: 1.858420  [ 6464/60000]
loss: 1.926321  [12864/60000]
loss: 1.736803  [19264/60000]
loss: 1.804929  [25664/60000]
loss: 1.780730  [32064/60000]
loss: 1.694075  [38464/60000]
loss: 1.799899  [44864/60000]
loss: 1.662251  [51264/60000]
loss: 1.627908  [57664/60000]
Test Error: 
 Accuracy: 71.8%, Avg loss: 1.617052 

Epoch 6
-------------------------------
loss: 1.629289  [   64/60000]
loss: 1.556617  [ 6464/60000]
loss

[codecarbon INFO @ 19:19:34] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:19:34] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:19:34] 0.000092 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 75.0%, Avg loss: 1.294680 

Epoch 7
-------------------------------
loss: 1.334487  [   64/60000]
loss: 1.230958  [ 6464/60000]
loss: 1.333256  [12864/60000]
loss: 1.135425  [19264/60000]
loss: 1.175402  [25664/60000]
loss: 1.138246  [32064/60000]
loss: 1.070066  [38464/60000]
loss: 1.233187  [44864/60000]
loss: 1.095757  [51264/60000]
loss: 1.058818  [57664/60000]
Test Error: 
 Accuracy: 79.0%, Avg loss: 1.036234 

Epoch 8
-------------------------------
loss: 1.102044  [   64/60000]
loss: 0.982730  [ 6464/60000]
loss: 1.077592  [12864/60000]
loss: 0.925333  [19264/60000]
loss: 0.954023  [25664/60000]
loss: 0.916697  [32064/60000]
loss: 0.859857  [38464/60000]
loss: 1.027258  [44864/60000]
loss: 0.923197  [51264/60000]
loss: 0.888429  [57664/60000]
Test Error: 
 Accuracy: 81.3%, Avg loss: 0.860073 

Epoch 9
-------------------------------
loss: 0.942075  [   64/60000]
loss: 0.815839  [ 6464/60000]
loss: 0.898712  [12864/60000]
loss: 0.783813  [19264/60000]
loss

[codecarbon INFO @ 19:19:49] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:19:49] Energy consumed for all CPUs : 0.000063 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:19:49] 0.000138 kWh of electricity used since the beginning.


loss: 0.687358  [19264/60000]
loss: 0.700657  [25664/60000]
loss: 0.671961  [32064/60000]
loss: 0.615327  [38464/60000]
loss: 0.780915  [44864/60000]
loss: 0.725539  [51264/60000]
loss: 0.701484  [57664/60000]


[codecarbon INFO @ 19:19:53] Energy consumed for RAM : 0.000080 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:19:53] Energy consumed for all CPUs : 0.000067 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:19:53] 0.000147 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 84.2%, Avg loss: 0.655574 

Done!
